## 데이터베이스를 기반으로 데이터를 학습시키는 방법

### CSV로 출력하고 CSV를 머신러닝 시스템에 전달하기
- 데이터를 쉽게 다룰 수 있다는 장점이 있으나 데이터가 깨지지 않게 주의해야함

## 키와 체중 데이터베이스 만들기
- 새로운 키와 체중을 데이터베이스에 추가하는 프로그램
- 데이터베이스의 값을 머신러닝 시스템에 학습시키는 프로그램
- 키와 체중을 입력하면 진단 결과를 출력하는 프로그램

### SQLite로 키와 체중 데이터베이스 만들기
- 키, 체중, 체형
    - 체형 : 0(저체중), 1(표준 체중) ~ 5(비만 4도)
- `CREATE TABLE` : 테이블 생성
- `sqlite3.connect()` : 데이터베이스 연결
- `execute()` : SQL 구문 실행

In [1]:
import sqlite3

dbpath = './db.sqlite3'
sql = '''
    CREATE TABLE IF NOT EXISTS person (
    id INTEGER PRIMARY KEY,
    height NUMBER,
    weight NUMBER,
    typeNo INTEGER
    )
'''

with sqlite3.connect(dbpath) as conn:
    conn.execute(sql)

### 새로운 키와 체중을 데이터베이스에 추가하기

In [2]:
import sqlite3
import random

dbpath = './db.sqlite3'

def insert_db(conn):
    # 더미 데이터 만들기
    height = random.randint(130, 180)
    weight = random.randint(30, 100)
    
    # 체형 데이터 생성
    type_no = 1
    bmi = weight / ((height / 100) ** 2)
    if bmi < 18.5:
        type_no = 0
    elif bmi < 25:
        type_no = 1
    elif bmi < 30:
        type_no = 2
    elif bmi < 35:
        type_no = 3
    elif bmi < 40:
        type_no = 4
    else:
        type_no = 5
    
    # 데이터베이스에 저장
    sql = '''
        INSERT INTO person (height, weight, typeNo)
        VALUES (?, ?, ?)
    '''
    
    values = (height, weight, type_no)
    conn.executemany(sql, [values])

# 100개 데이터 삽입
with sqlite3.connect(dbpath) as conn:
    for i in range(100):
        insert_db(conn)
    
    # 확인
    c = conn.execute('SELECT count(*) FROM person')
    cnt = c.fetchone()
    print(cnt[0])

100


In [3]:
import sqlite3

dbpath = './db.sqlite3'
select_sql = 'SELECT * FROM person'
with sqlite3.connect(dbpath) as conn:
    i = 0
    for row in conn.execute(select_sql):
        print(row)
        if i == 5:
            break
        i += 1

(1, 170, 61, 1)
(2, 166, 75, 2)
(3, 138, 48, 2)
(4, 175, 66, 1)
(5, 176, 76, 1)
(6, 135, 60, 3)


## 키, 체중, 체형 학습하기

In [4]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

in_size = 2 # 체중, 키
nb_classes = 6

# MLP 모델 정의
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(in_size, )))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

# 컴파일
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(), metrics=['accuracy'])

model.save('../models/hw_model.h5')
print('saved')

saved


#### 100개 데이터 추출해 학습

In [5]:
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import sqlite3
import os

# 데이터 읽기
dbpath = './db.sqlite3'
select_sql = 'SELECT * FROM person ORDER BY id DESC LIMIT 100'

# 데이터를 리스트에 추가
x = []
y = []
with sqlite3.connect(dbpath) as conn:
    for row in conn.execute(select_sql):
        id, height, weight, type_no = row
        # 데이터 정규화
        height = height / 200
        weight = weight / 150
        y.append(type_no)
        x.append(np.array([height, weight]))
    
# 모델 읽기
model = load_model('../models/hw_model.h5')

# 이미 학습 데이터가 있는 경우 읽어들이기
if os.path.exists('../models/hw_weights.h5'):
    model.load_weights('../models/hw_weights.h5')
    
nb_classes = 6
y = to_categorical(y, nb_classes) # OHE 변환

# 학습
model.fit(np.array(x), y, batch_size=50, epochs=100)

# 결과 저장
model.save_weights('../models/hw_weights.h5')

Epoch 1/100
2/2 [==============================] - 1s 5ms/step - loss: 1.7873 - accuracy: 0.1200
Epoch 2/100
2/2 [==============================] - 0s 4ms/step - loss: 1.7589 - accuracy: 0.2200
Epoch 3/100
2/2 [==============================] - 0s 5ms/step - loss: 1.7579 - accuracy: 0.2300
Epoch 4/100
2/2 [==============================] - 0s 4ms/step - loss: 1.7404 - accuracy: 0.2500
Epoch 5/100
2/2 [==============================] - 0s 4ms/step - loss: 1.7323 - accuracy: 0.2300
Epoch 6/100
2/2 [==============================] - 0s 5ms/step - loss: 1.7340 - accuracy: 0.2800
Epoch 7/100
2/2 [==============================] - 0s 4ms/step - loss: 1.7231 - accuracy: 0.2600
Epoch 8/100
2/2 [==============================] - 0s 4ms/step - loss: 1.7225 - accuracy: 0.2600
Epoch 9/100
2/2 [==============================] - 0s 5ms/step - loss: 1.7218 - accuracy: 0.2300
Epoch 10/100
2/2 [==============================] - 0s 4ms/step - loss: 1.7193 - accuracy: 0.2500
Epoch 11/100
2/2 [===========

2/2 [==============================] - 0s 3ms/step - loss: 1.5048 - accuracy: 0.3600
Epoch 85/100
2/2 [==============================] - 0s 6ms/step - loss: 1.5036 - accuracy: 0.3500
Epoch 86/100
2/2 [==============================] - 0s 4ms/step - loss: 1.4962 - accuracy: 0.3700
Epoch 87/100
2/2 [==============================] - 0s 5ms/step - loss: 1.4858 - accuracy: 0.3900
Epoch 88/100
2/2 [==============================] - 0s 4ms/step - loss: 1.4867 - accuracy: 0.3600
Epoch 89/100
2/2 [==============================] - 0s 6ms/step - loss: 1.4744 - accuracy: 0.3900
Epoch 90/100
2/2 [==============================] - 0s 3ms/step - loss: 1.4864 - accuracy: 0.3400
Epoch 91/100
2/2 [==============================] - 0s 5ms/step - loss: 1.4767 - accuracy: 0.4100
Epoch 92/100
2/2 [==============================] - 0s 3ms/step - loss: 1.4655 - accuracy: 0.3800
Epoch 93/100
2/2 [==============================] - 0s 4ms/step - loss: 1.4642 - accuracy: 0.4600
Epoch 94/100
2/2 [===============

### Keras의 fit() 메서드는 새로운 데이터를 추가로 학습할 수 있음
- 이미 학습된 데이터가 있는 상태에서 model.fit()을 호출하면 이전에 학습된 결과에 추가로 새로운 데이터가 학습됨

### 정답률 확인하기

In [6]:
from tensorflow.keras.models import load_model
import numpy as np

# 학습 모델 읽어 들이기
model = load_model('../models/hw_model.h5')

# 학습한 데이터 읽어 들이기
model.load_weights('../models/hw_weights.h5')

# 레이블
LABELS = [
    '저체중', '표준 체중', '비만(1도)',
    '비만(2도)', '비만(3도)', '비만(4도)'
]

# 테스트 데이터
height = 160
weight = 50

# 정규화
text_x = [height / 200, weight / 150]

# 예측
pre = model.predict(np.array([text_x]))
idx = pre[0].argmax()
print(LABELS[idx], '/ 가능성', pre[0][idx])

표준 체중 / 가능성 0.2930612


## 데이터 추가하여 학습

In [7]:
# 데이터 4900개 추가 삽입
with sqlite3.connect(dbpath) as conn:
    for i in range(4900):
        insert_db(conn)
    
    # 확인
    c = conn.execute('SELECT count(*) FROM person')
    cnt = c.fetchone()
    print(cnt[0])

5000


In [8]:
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import sqlite3
import os

# 데이터 읽기
dbpath = './db.sqlite3'
select_sql = 'SELECT * FROM person ORDER BY id DESC LIMIT 5000'

# 데이터를 리스트에 추가
x = []
y = []
with sqlite3.connect(dbpath) as conn:
    for row in conn.execute(select_sql):
        id, height, weight, type_no = row
        # 데이터 정규화
        height = height / 200
        weight = weight / 150
        y.append(type_no)
        x.append(np.array([height, weight]))
    
# 모델 읽기
model = load_model('../models/hw_model.h5')

# 이미 학습 데이터가 있는 경우 읽어들이기
if os.path.exists('../models/hw_weights.h5'):
    model.load_weights('../models/hw_weights.h5')
    
nb_classes = 6
y = to_categorical(y, nb_classes) # OHE 변환

# 학습
model.fit(np.array(x), y, batch_size=50, epochs=100)

# 결과 저장
model.save_weights('../models/hw_weights.h5')

Epoch 1/100
100/100 [==============================] - 0s 2ms/step - loss: 1.2892 - accuracy: 0.5346
Epoch 2/100
100/100 [==============================] - 0s 2ms/step - loss: 1.1465 - accuracy: 0.5732
Epoch 3/100
100/100 [==============================] - 0s 2ms/step - loss: 1.0427 - accuracy: 0.6038
Epoch 4/100
100/100 [==============================] - 0s 2ms/step - loss: 0.9660 - accuracy: 0.6292
Epoch 5/100
100/100 [==============================] - 0s 2ms/step - loss: 0.9027 - accuracy: 0.6486
Epoch 6/100
100/100 [==============================] - 0s 2ms/step - loss: 0.8555 - accuracy: 0.6556
Epoch 7/100
100/100 [==============================] - 0s 2ms/step - loss: 0.8119 - accuracy: 0.6698
Epoch 8/100
100/100 [==============================] - 0s 2ms/step - loss: 0.7695 - accuracy: 0.6950
Epoch 9/100
100/100 [==============================] - 0s 2ms/step - loss: 0.7356 - accuracy: 0.7028
Epoch 10/100
100/100 [==============================] - 0s 2ms/step - loss: 0.7052 - accura

100/100 [==============================] - 0s 2ms/step - loss: 0.2031 - accuracy: 0.9300
Epoch 82/100
100/100 [==============================] - 0s 2ms/step - loss: 0.2017 - accuracy: 0.9302
Epoch 83/100
100/100 [==============================] - 0s 2ms/step - loss: 0.2019 - accuracy: 0.9306
Epoch 84/100
100/100 [==============================] - 0s 2ms/step - loss: 0.1933 - accuracy: 0.9340
Epoch 85/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2005 - accuracy: 0.9276
Epoch 86/100
100/100 [==============================] - 0s 3ms/step - loss: 0.1894 - accuracy: 0.9378
Epoch 87/100
100/100 [==============================] - 0s 3ms/step - loss: 0.1957 - accuracy: 0.9294
Epoch 88/100
100/100 [==============================] - 0s 2ms/step - loss: 0.1904 - accuracy: 0.9394
Epoch 89/100
100/100 [==============================] - 0s 2ms/step - loss: 0.1977 - accuracy: 0.9294
Epoch 90/100
100/100 [==============================] - 0s 2ms/step - loss: 0.1889 - accuracy: 

In [9]:
from tensorflow.keras.models import load_model
import numpy as np

# 학습 모델 읽어 들이기
model = load_model('../models/hw_model.h5')

# 학습한 데이터 읽어 들이기
model.load_weights('../models/hw_weights.h5')

# 레이블
LABELS = [
    '저체중', '표준 체중', '비만(1도)',
    '비만(2도)', '비만(3도)', '비만(4도)'
]

# 테스트 데이터
height = 160
weight = 50

# 정규화
text_x = [height / 200, weight / 150]

# 예측
pre = model.predict(np.array([text_x]))
idx = pre[0].argmax()
print(LABELS[idx], '/ 가능성', pre[0][idx])

표준 체중 / 가능성 0.9768397


### 분류 정답률 확인하기

In [11]:
from tensorflow.keras.models import load_model
import numpy as np

# 학습 모델 읽어 들이기
model = load_model('../models/hw_model.h5')

# 학습한 데이터 읽어 들이기
model.load_weights('../models/hw_weights.h5')

x = []
y = []
for i in range(1000):
    h = random.randint(130, 180)
    w = random.randint(30, 100)
    type_no = 1
    
    bmi = w / ((h / 100) ** 2)
    if bmi < 18.5:
        type_no = 0
    elif bmi < 25:
        type_no = 1
    elif bmi < 30:
        type_no = 2
    elif bmi < 35:
        type_no = 3
    elif bmi < 40:
        type_no = 4
    else:
        type_no = 5
    x.append(np.array([h/200, w/150]))
    y.append(type_no)
    
x = np.array(x)
y = to_categorical(y, 6)

score = model.evaluate(x, y, verbose=1)
print('정답률 =', score[1], '손실률 :', score[0])

32/32 [==============================] - 0s 1ms/step - loss: 0.1569 - accuracy: 0.9630
정답률 = 0.9629999995231628 손실률 : 0.1569320410490036
